In [22]:
import pandas as pd
import bokeh.io
path_to_incubations = "../lab_work/sessions/incubations/"
from GCPDFReader.server.data_viz import create_co2_chart, create_methane_chart, get_methane_df
from GCPDFReader.server.extract_incubation_data import extract_and_process_incubation_data
from GCPDFReader.server.transform import process_sample_data

bokeh.io.output_notebook()

csv_path = "csvs/tidied_1693791964.csv"
data_df = pd.read_csv(f"GCPDFReader/{csv_path}")
incubation_df = extract_and_process_incubation_data(path_to_incubations)

df = process_sample_data(data_df, incubation_df)


plot = create_co2_chart(df)
bokeh.io.show(plot)

Loading BokehJS ...

ValueError: cannot convert float NaN to integer

In [23]:
from GCPDFReader.server.data_viz import create_methane_chart
plot = create_methane_chart(df)
bokeh.io.show(plot)
# df["incubation_length"].max()

In [52]:
temp = np.array([[142828.00, 20000.00],
[143498.00, 20000.00],
[141214.00, 20000.00],
[140473.00, 20000.00],
[142588.00, 20000.00],
[97.75, 2.18],
[57.81, 2.18],
[69.16, 2.18],
[88.26, 2.18]])
G = np.vstack([np.ones(temp.shape[0]), temp[:, 0]]).T
d = temp[:, 1]
m = np.linalg.lstsq(G, d)[0]
print(m)
test_areas = np.array([100, 1e3, 1e4, 1e5])
cov = np.linalg.inv(G.T @ G)
testG = np.vstack([np.ones(test_areas.shape[0]), test_areas]).T
print(np.sqrt(np.diag(testG @ cov @ testG.T))) 

[-7.30984444  0.1407688 ]
[0.49990437 0.49674508 0.46610686 0.34778542]


/var/folders/f3/zyvmf_xs32qbkl_h7913df540000gn/T/ipykernel_68169/2481762116.py:12: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  m = np.linalg.lstsq(G, d)[0]


In [7]:
import bokeh.io
from bokeh import plotting
from bokeh.io import export_png
from bokeh.models import ColumnDataSource, ColorBar, LinearColorMapper, CategoricalColorMapper
from bokeh.palettes import Spectral6, Inferno256
from bokeh.transform import jitter

import pandas as pd
import numpy as np

def time_v_conc(title, df):
    incubation_length_max = int(df["incubation_length"].max())
    wet_df = ColumnDataSource(df[df["treatment"]=="Wet"])
    dry_df = ColumnDataSource(df[df["treatment"]=="Dry"])
    # ratios = ["1:0", "16:1", "1:1", "1:5"]
    # exp_cmap = LinearColorMapper(palette=Inferno256, low=incubation_length_max, high=0)
    clr_mapper = CategoricalColorMapper(palette=["red", "blue", "green", "purple"], factors=["1:0", "16:1", "1:1", "1:5"])


    TOOLTIPS = [
    ("sample_id", "@sample_id"),
    ("calculated_conc", "@calculated_conc")
        ]
    
    p = plotting.figure(
        title=title,
        x_axis_label = "Incubation Length",
        y_axis_label = "Calculated Conc (ppm)",
        y_axis_type="log",
        y_range=[10e-2, 10e5],
        x_range=[0, incubation_length_max],
        toolbar_location='above',
        tooltips = TOOLTIPS,
        sizing_mode="stretch_width"
    )
    p.triangle(
        source=wet_df,
        x = jitter("incubation_length", width=0.5, range=p.x_range),
        y = "calculated_conc",
        color={"field":"salt_ratio", "transform":clr_mapper},
        size=10,
        )
    p.circle(
        source=dry_df,
                x = jitter("incubation_length", width=0.5, range=p.x_range),
        y = "calculated_conc",
        color={"field":"salt_ratio", "transform":clr_mapper},
        size=10,
        )
   

    p.add_layout(bokeh.models.Legend(), "right")
    p.legend.click_policy = "hide"
    return p
    


In [10]:
methane_df = get_methane_df(df)
hold = time_v_conc("Anything", methane_df)
bokeh.io.show(hold)

In [6]:
methane_df = get_methane_df(df)
hold = time_v_conc("Anything", methane_df)
bokeh.io.show(hold)

NameError: name 'time_v_conc' is not defined

In [20]:
methane_df[['Time', 'calculated_conc', 'incubation_length', 'sample_id']].sort_values(by='incubation_length')

,Time,calculated_conc,incubation_length,sample_id
11,2.565,92.689336,35.0,40mL_2.0
13,2.567,86.148035,35.0,40mL_2.0
15,2.569,33.246407,35.0,40mL_2.1
17,2.567,30.798740,35.0,40mL_2.1
19,2.567,52.335823,35.0,40mL_2.2
21,2.568,52.439167,35.0,40mL_2.2
122,NaN,1.000000,35.0,40mL_2.3
120,NaN,1.000000,51.0,40mL_1.4
118,NaN,1.000000,51.0,40mL_1.3
116,NaN,1.000000,51.0,40mL_1.2
